In [17]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

/disk2/fli/REVIVAL2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys
import os
import numpy
import openmm
print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"OpenMM: {openmm.version.version}")
print(f"Environment: {os.environ}")

Python: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
NumPy: 2.0.2
OpenMM: 7.7.0.dev-130124a
Environment: environ({'USER': 'fli', 'SSH_CLIENT': '131.215.250.154 52071 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'MOTD_SHOWN': 'pam', 'HOME': '/disk2/fli', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1002/bus', 'LOGNAME': 'fli', '_': '/disk2/fli/miniconda3/envs/vina/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '11757', 'VSCODE_CLI_REQUIRE_TOKEN': '4074d282-3f3d-4e21-a716-9309c4a37c30', 'PATH': '/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/.vscode-server/cli/servers/Stable-91fbdddc47bc9c09064bf7acf133d22631cbf083/server/bin/remote-cli:/disk2/fli/.local/bin:/disk2/fli/openmpi/bin:/disk2/fli/orca_6_0_1:/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/miniconda3/condabin:/disk2/fli/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:

In [25]:
from REVIVAL.zs.vina import dock

dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V/I165A:I183A:Y301V_0.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is model)

1 molecule converted


Detected residues to replace: set()


1 molecule converted


Formatting ligand 4bromo
Checking for ions...
Processing ligand 4bromo from zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0.pdb
insdie extract_substruct
/disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/4bromo/4bromo_temp.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C6>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C7>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode

1 molecule converted


Merging zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0.pdbqt
Merging zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/PLP-dependent_aminoacrylate/PLP-dependent_aminoacrylate.pdbqt
Merging zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/Na+/NA.pdbqt
Combined ATOM lines saved to zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0_cofactors/I165A:I183A:Y301V_0_cofactors.pdbqt
Vina run (score-only) completed successfully. Log saved to: zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0-4bromo-substrate-score_only_log.txt


'zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0-4bromo-substrate-score_only_log.txt'

In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdmolfiles import PDBWriter, MolFromPDBFile

def optimize_hydrogens(input_pdb, output_pdb):
    """
    Optimize the positions of hydrogens in a PDB file without altering heavy atom coordinates.

    Args:
        input_pdb (str): Path to the input PDB file.
        output_pdb (str): Path to save the optimized PDB file.
        forcefield (str): Force field to use for optimization ("MMFF" or "UFF").
    """
    # Step 1: Load the PDB file
    mol = MolFromPDBFile(input_pdb, sanitize=False, removeHs=False)
    if mol is None:
        raise ValueError(f"Failed to load PDB file: {input_pdb}")

    # Step 2: Add missing hydrogens (RDKit will automatically place them)
    mol = Chem.AddHs(mol)

    # Step 3: Set up force field
    ff = AllChem.UFFGetMoleculeForceField(mol)

    # Step 4: Constrain heavy atom positions
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() > 1:  # Heavy atoms (C, N, O, etc.)
            ff.AddFixedPoint(atom.GetIdx())

    # Step 5: Minimize hydrogen positions
    ff.Minimize()

    # Step 6: Save optimized structure to a new PDB file
    with PDBWriter(output_pdb) as writer:
        writer.write(mol)

    print(f"Optimized hydrogen positions saved to: {output_pdb}")

In [12]:

# Example usage
optimize_hydrogens(
    input_pdb="/disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/4bromo/4bromo_test.pdb",
    output_pdb="/disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/4bromo/4bromo_test_opt.pdb",
)


Optimized hydrogen positions saved to: /disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/4bromo/4bromo_test_opt.pdb


[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 0
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (0)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 1
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (1)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 2
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (2)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 3
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (3)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 4
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (4)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 5
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (5)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 6
[18:06:20] UFFTYPER: Unrecognized atom type: N_ (6)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 7
[18:06:20] UFFTYPER: Unrecognized atom type: C_ (7)
[18:06:20] UFFTYPER: Unrecognized hybridization for atom: 8
[18:06:20] UFFTYPER: Unrecognized atom type:

In [15]:
!obabel /disk2/fli/REVIVAL2/zs/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V/I165A:I183A:Y301V_0.cif -O /disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0_2.pdb

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is model)

1 molecule converted
